In [1]:
from web3 import Web3, IPCProvider

geth_ipc = '/media/Warehouse/Warehouse/Ethereum/data/.ethereum/geth.ipc'
w3 = Web3(IPCProvider(geth_ipc))
# Check
print ("Latest Ethereum block number" , w3.eth.blockNumber)

Latest Ethereum block number 16535422


In [2]:
import pickle

import pandas as pd
import tqdm

In [3]:
tr_data = pd.read_csv('data/merged_221220_221231_with_contracts.csv', usecols=['tx_hash'])
tr_data.head()

,tx_hash
0,0x14e970dba71ef93b42f337d3494c4d6e8fee4d83ce38...
1,0x33fcc824c05553a3ec31a7d71dc6dbb26cf51d608009...
2,0x555ffb08812804e23fe18467886edaa167f502488c6b...
3,0x9febfdc63677cd0decb532cb049004add2959d7f1a1f...
4,0xe24c2fb8e8d581d86710a0ca685e27ed6c36ca722d8e...


In [ ]:
# Получим данные о газе для каждой транзакции

additional_tx_data = {}

for i, tx_hash in tqdm.tqdm(enumerate(tr_data['tx_hash']), total=len(tr_data)):
    tx = w3.eth.get_transaction(tx_hash)
    tx_receipt = w3.eth.get_transaction_receipt(tx_hash)
    additional_tx_data[tx_hash] = {
        'maxFeePerGas': tx.get('maxFeePerGas', 0),
        'maxPriorityFeePerGas': tx.get('maxPriorityFeePerGas', 0),
        'gasUsed': tx_receipt['gasUsed'],
        'effectiveGasPrice': tx_receipt['effectiveGasPrice']
    }
    if i % 100000 == 0:
        with open('additional_tx_data.pickle', 'wb') as f:
            pickle.dump(additional_tx_data, f)

 92%|███████████████████████████████████████████████████████████▉     | 10639182/11527423 [11:50:16<47:04, 314.51it/s]

In [9]:
with open('additional_tx_data.pickle', 'wb') as f:
            pickle.dump(additional_tx_data, f)

In [12]:
for i, tx_hash in tqdm.tqdm(enumerate(tr_data['tx_hash']), total=len(tr_data)):
    if tx_hash in additional_tx_data:
        continue
    tx = w3.eth.get_transaction(tx_hash)
    tx_receipt = w3.eth.get_transaction_receipt(tx_hash)
    additional_tx_data[tx_hash] = {
        'maxFeePerGas': tx.get('maxFeePerGas', 0),
        'maxPriorityFeePerGas': tx.get('maxPriorityFeePerGas', 0),
        'gasUsed': tx_receipt['gasUsed'],
        'effectiveGasPrice': tx_receipt['effectiveGasPrice']
    }
with open('additional_tx_data.pickle', 'wb') as f:
    pickle.dump(additional_tx_data, f)

100%|████████████████████████████████████████████████████████████████| 11527423/11527423 [00:04<00:00, 2753343.53it/s]


In [8]:
# Получим nonce

additional_tx_data = {}

for i, tx_hash in tqdm.tqdm(enumerate(tr_data['tx_hash']), total=len(tr_data)):
    tx = w3.eth.get_transaction(tx_hash)
    tx_receipt = w3.eth.get_transaction_receipt(tx_hash)
    additional_tx_data[tx_hash] = {
        'nonce': tx['nonce']
    }
    if i % 100000 == 0:
        with open('additional_tx_data_nonce.pickle', 'wb') as f:
            pickle.dump(additional_tx_data, f)

100%|█████████████████████████████████████████████████████████████████| 11527306/11527306 [12:33:09<00:00, 255.09it/s]


In [9]:
with open('additional_tx_data_nonce.pickle', 'wb') as f:
            pickle.dump(additional_tx_data, f)